In [1]:
import pandas as pd

In [2]:
df =pd.read_json('/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/mol2patentfiltered.jsonl',lines=True)

: 

In [ ]:
df

In [ ]:
df.head

In [ ]:
import ijson

# File path
file_path = '/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patcid_patent_to_molecules_uspto.jsonl'



# Read a small portion of the file
with open(file_path, 'r') as file:
    # Adjust the size to fetch an appropriate portion
    chunk = file.read(1024 * 1024)  # Read the first 1MB

    # Find where the JSON starts and ends in this chunk
    start = chunk.find('[')  # Find the start of the JSON array
    end = chunk.find(']', start) + 1  # Find the end of the JSON array

    # Parse only the valid JSON portion
    data = json.loads(chunk[start:end])

    # Display the first few items
    for i, item in enumerate(data[:5]):
        print(f"Item {i+1}: {item}\n")


: 

In [3]:
import requests
from bs4 import BeautifulSoup

# Define APIs to test and their example parameters
apis = {
    "Espacenet": {
        "url": "https://ops.epo.org/rest-services/published-data/publication/epodoc/{patent_id}/classification/cpc",
        "headers": {"Authorization": "Bearer YOUR_API_KEY"},  # Replace with your API key
        "example_patent_id": "EP1000000"
    },
    "USPTO PatentsView": {
        "url": "https://api.patentsview.org/patents/query?q={\"patent_number\":\"{patent_id}\"}&f=[\"patent_id\"]",
        "headers": {},
        "example_patent_id": "US20200136057"
    },
    "WIPO PatentScope": {
        "url": "https://patentscope.wipo.int/search/en/detail.jsf?docId={patent_id}&office=WO",
        "headers": {},
        "example_patent_id": "WO2020012345"
    },
    "Lens.org": {
        "url": "https://api.lens.org/lens/patents/{patent_id}",
        "headers": {"Authorization": "Bearer YOUR_API_KEY"},  # Replace with your API key
        "example_patent_id": "US20200136057"
    }
}

# Function to test API endpoints
def test_apis(apis):
    results = {}
    for api_name, api_info in apis.items():
        print(f"Testing {api_name} API...")
        url = api_info["url"].replace("{patent_id}", api_info["example_patent_id"])
        try:
            response = requests.get(url, headers=api_info.get("headers", {}))
            if response.status_code == 200:
                if api_name == "WIPO PatentScope":
                    # Parse HTML response for WIPO
                    soup = BeautifulSoup(response.text, "html.parser")
                    cpc_data = soup.find_all("desired_element")  # Replace 'desired_element' with actual tag or class
                    results[api_name] = {
                        "status": "Success",
                        "response_sample": cpc_data
                    }
                else:
                    results[api_name] = {
                        "status": "Success",
                        "response_sample": response.json() if "application/json" in response.headers.get("Content-Type", "") else response.text
                    }
            else:
                results[api_name] = {
                    "status": "Failed",
                    "error": f"HTTP {response.status_code}: {response.text}"
                }
        except Exception as e:
            results[api_name] = {
                "status": "Error",
                "error": str(e)
            }
    return results

# Run tests
results = test_apis(apis)

# Display results
for api_name, result in results.items():
    print(f"\nAPI: {api_name}")
    print(f"Status: {result['status']}")
    if result['status'] == "Success":
        print(f"Response Sample: {result['response_sample'][:500]}...")  # Truncate for display
    else:
        print(f"Error: {result['error']}")


Testing Espacenet API...
Testing USPTO PatentsView API...
Testing WIPO PatentScope API...
Testing Lens.org API...

API: Espacenet
Status: Failed
Error: HTTP 400: <error><code>400</code><message>invalid_access_token</message><description>Access token is invalid</description>
				</error>
			

API: USPTO PatentsView
Status: Success


TypeError: unhashable type: 'slice'

In [4]:
import requests
from bs4 import BeautifulSoup

# Define APIs to test and their example parameters
apis = {
    "Espacenet": {
        "url": "https://ops.epo.org/rest-services/published-data/publication/epodoc/{patent_id}/classification/cpc",
        "headers": {"Authorization": "Bearer YOUR_API_KEY"},  # Replace with your API key
        "example_patent_id": "EP1000000"
    },
    "USPTO PatentsView": {
        "url": "https://api.patentsview.org/patents/query?q={\"patent_number\":\"{patent_id}\"}&f=[\"patent_id\"]",
        "headers": {},
        "example_patent_id": "US20200136057"
    },
    "WIPO PatentScope": {
        "url": "https://patentscope.wipo.int/search/en/detail.jsf?docId={patent_id}&office=WO",
        "headers": {},
        "example_patent_id": "WO2020012345"
    },
    "Lens.org": {
        "url": "https://api.lens.org/lens/patents/{patent_id}",
        "headers": {"Authorization": "Bearer YOUR_API_KEY"},  # Replace with your API key
        "example_patent_id": "US20200136057"
    }
}

# Function to test API endpoints
def test_apis(apis):
    results = {}
    for api_name, api_info in apis.items():
        print(f"Testing {api_name} API...")
        url = api_info["url"].replace("{patent_id}", api_info["example_patent_id"])
        try:
            response = requests.get(url, headers=api_info.get("headers", {}))
            if response.status_code == 200:
                if api_name == "WIPO PatentScope":
                    # Parse HTML response for WIPO
                    soup = BeautifulSoup(response.text, "html.parser")
                    cpc_data = soup.find_all("desired_element")  # Replace 'desired_element' with actual tag or class
                    results[api_name] = {
                        "status": "Success",
                        "response_sample": str(cpc_data)[:500]  # Truncate for display
                    }
                else:
                    response_data = response.json() if "application/json" in response.headers.get("Content-Type", "") else response.text
                    results[api_name] = {
                        "status": "Success",
                        "response_sample": str(response_data)[:500]  # Truncate for display
                    }
            else:
                results[api_name] = {
                    "status": "Failed",
                    "error": f"HTTP {response.status_code}: {response.text}"
                }
        except Exception as e:
            results[api_name] = {
                "status": "Error",
                "error": str(e)
            }
    return results

# Run tests
results = test_apis(apis)

# Display results
for api_name, result in results.items():
    print(f"\nAPI: {api_name}")
    print(f"Status: {result['status']}")
    if result['status'] == "Success":
        print(f"Response Sample: {result['response_sample']}...")  # Truncated response
    else:
        print(f"Error: {result['error']}")


Testing Espacenet API...
Testing USPTO PatentsView API...
Testing WIPO PatentScope API...
Testing Lens.org API...

API: Espacenet
Status: Failed
Error: HTTP 400: <error><code>400</code><message>invalid_access_token</message><description>Access token is invalid</description>
				</error>
			

API: USPTO PatentsView
Status: Success
Response Sample: {'patents': None, 'count': 0, 'total_patent_count': 0}...

API: WIPO PatentScope
Status: Success
Response Sample: []...

API: Lens.org
Status: Failed
Error: HTTP 500: {"reference":"b8f72f2b-5227-4781-b0bd-b10deed59a8b","message":"No product in scope or unknown endpoint '/lens/patents/US20200136057'","code":500}


In [7]:
from tqdm import tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup

# File containing the list of patent IDs
patent_ids_file = "/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_ids.txt"
output_file = "/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_cpc_info.txt"

# Function to get CPC codes from WIPO PatentScope API
def get_cpc_codes(patent_ids_file, output_file):
    base_url = "https://patentscope.wipo.int/search/en/detail.jsf?docId={patent_id}&office=WO"

    with open(patent_ids_file, 'r') as file:
        patent_ids = [line.strip() for line in file.readlines()]

    with open(output_file, 'w') as output:
        for patent_id in tqdm(patent_ids):
            print(f"Fetching CPC codes for Patent ID: {patent_id}...")
            if "US" in str(patent_id):
                try:
                    url = base_url.replace("{patent_id}", patent_id)
                    response = requests.get(url)

                    if response.status_code == 200:
                        soup = BeautifulSoup(response.text, "html.parser")

                        # Extract CPC data from the HTML response (adjust selector as needed)
                        cpc_elements = soup.select(".cpc-classification")  # Example CSS selector
                        cpc_codes = [element.text.strip() for element in cpc_elements]

                        if cpc_codes:
                            output.write(f"{patent_id}: {', '.join(cpc_codes)}\n")
                            print(f"CPC Codes: {', '.join(cpc_codes)}")
                        else:
                            output.write(f"{patent_id}: No CPC codes found\n")
                            print("No CPC codes found.")
                    else:
                        output.write(f"{patent_id}: Failed to fetch (HTTP {response.status_code})\n")
                        print(f"Failed to fetch data (HTTP {response.status_code}).")

                except Exception as e:
                    output.write(f"{patent_id}: Error ({str(e)})\n")
                    print(f"Error fetching data: {str(e)}")
            else:
                next

# Run the function
get_cpc_codes(patent_ids_file, output_file)

print(f"CPC codes have been saved to {output_file}.")


In [12]:
import requests
from bs4 import BeautifulSoup
import time

# File containing the list of patent IDs
patent_ids_file = "/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_ids.txt"
output_file = "/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_cpc_info.txt"

# Function to get CPC codes from WIPO PatentScope API
def get_cpc_codes(patent_ids_file, output_file):
    base_url = "https://patentscope.wipo.int/search/en/detail.jsf?docId={patent_id}&office=WO"

    with open(patent_ids_file, 'r') as file:
        patent_ids = [line.strip() for line in file.readlines()]

    with open(output_file, 'w') as output:
        for patent_id in patent_ids:
            print(f"Fetching CPC codes for Patent ID: {patent_id}...")
            try:
                url = base_url.replace("{patent_id}", patent_id)
                response = requests.get(url)

                # Log response for debugging
                with open(f"response_{patent_id}.html", 'w', encoding='utf-8') as html_file:
                    html_file.write(response.text)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, "html.parser")

                    # Extract CPC data from the HTML response (adjust selector as needed)
                    cpc_elements = soup.select(".cpc-classification")  # Example CSS selector
                    cpc_codes = [element.text.strip() for element in cpc_elements]

                    if cpc_codes:
                        output.write(f"{patent_id}: {', '.join(cpc_codes)}\n")
                        print(f"CPC Codes: {', '.join(cpc_codes)}")
                    else:
                        output.write(f"{patent_id}: No CPC codes found\n")
                        print("No CPC codes found.")
                else:
                    output.write(f"{patent_id}: Failed to fetch (HTTP {response.status_code})\n")
                    print(f"Failed to fetch data (HTTP {response.status_code}).")

            except Exception as e:
                output.write(f"{patent_id}: Error ({str(e)})\n")
                print(f"Error fetching data: {str(e)}")

            # Add a delay to avoid getting blocked by the server
            time.sleep(1)

# Run the function
get_cpc_codes(patent_ids_file, output_file)

print(f"CPC codes have been saved to {output_file}.")


Fetching CPC codes for Patent ID: CN101010063...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101010063B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101027318...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101027318B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101056895...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101056895B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101080480...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101080480B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101087611...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101087611B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101100411...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101100411B...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101111506...
No CPC codes found.
Fetching CPC codes for Patent ID: CN101111506B...
No CPC codes found.
Fetching CPC codes for Pate

KeyboardInterrupt: 

In [6]:
import requests
import time

def get_primary_cpc(patent_id):
    """Simple API call to get CPC code"""
    base_url = "https://api.patentsview.org/patents/query"
    query = {
        "q": {"patent_id": patent_id},
        "f": ["patent_id", "cpc_section_id", "cpc_subsection_id", "cpc_group_id", "cpc_subgroup_id"],
    }
    
    response = requests.post(base_url, json=query)
    if response.status_code == 200:
        data = response.json()
        if data['patents']:
            patent = data['patents'][0]
            return f"{patent['cpc_section_id']}{patent['cpc_subsection_id']}{patent['cpc_group_id']}/{patent['cpc_subgroup_id']}"
    return None

# Read first 1000 patent IDs
with open('/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_ids.txt', 'r') as f:
    patent_ids = [line.strip() for line in f][:1000]

results = {}
attempts = 0
successful_matches = 0

# Process patents
for patent_id in patent_ids:
    # Clean patent ID
    if patent_id.startswith('US'):
        clean_id = patent_id[2:]  # Remove 'US'
        if any(c.isalpha() for c in clean_id[-2:]):
            clean_id = clean_id[:-2]  # Remove last two chars if letters present
            
        # Get CPC and print every 100th attempt
        attempts += 1
        if attempts % 100 == 0:
            print(f"\nAttempt {attempts}")
            print(f"Patent ID: {patent_id} -> {clean_id}")
            
        cpc = get_primary_cpc(clean_id)
        
        if cpc:
            successful_matches += 1
            results[patent_id] = cpc
            print(f"Found CPC #{successful_matches}: {patent_id} -> {cpc}")
            
            # Stop if we have 100 matches
            if successful_matches >= 100:
                print("Success: Found 100 CPC codes")
                break
                
    # Stop if too many attempts
    if attempts >= 1000:
        print("Failed: 1000 attempts reached")
        break
        
    time.sleep(0.1)  # Rate limiting

print(f"\nFinal Summary:")
print(f"Attempts: {attempts}")
print(f"Matches: {successful_matches}")


Final Summary:
Attempts: 0
Matches: 0


In [7]:
import requests

# Test with one patent ID
test_id = "10000272"  # This is the cleaned version of USRE048226

# Make API call
url = "https://api.patentsview.org/patents/query"
query = {
    "q": {"patent_id": test_id},
    "f": ["patent_id", "cpc_section_id", "cpc_subsection_id", "cpc_group_id", "cpc_subgroup_id"]
}

print("Making API call...")
print("Query:", query)

response = requests.post(url, json=query)
print(f"Status code: {response.status_code}")
print("\nResponse:")
print(response.json())

Making API call...
Query: {'q': {'patent_id': '10000272'}, 'f': ['patent_id', 'cpc_section_id', 'cpc_subsection_id', 'cpc_group_id', 'cpc_subgroup_id']}
Status code: 200

Response:
{'patents': [{'patent_id': '10000272', 'cpcs': [{'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B5/02'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B5/024'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B5/18'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B27/065'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B27/12'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B27/16'}, {'cpc_section_id': 'B', 'cpc_subsection_id': 'B32', 'cpc_group_id': 'B32B', 'cpc_subgroup_id': 'B32B27/288'}, {'cpc_se

In [8]:
import requests

# Test with one patent ID
test_id = "10000272"

# Make API call
url = "https://api.patentsview.org/patents/query"
query = {
    "q": {"patent_id": test_id},
    "f": ["patent_id", "cpc_section_id", "cpc_subsection_id", "cpc_group_id", "cpc_subgroup_id"]
}

response = requests.post(url, json=query)
data = response.json()

# Extract primary CPC code (first one in the list)
if data['patents'] and data['patents'][0]['cpcs']:
    primary_cpc = data['patents'][0]['cpcs'][0]
    cpc_code = f"{primary_cpc['cpc_section_id']}{primary_cpc['cpc_subsection_id']}{primary_cpc['cpc_group_id']}/{primary_cpc['cpc_subgroup_id']}"
    print(f"Patent {test_id} primary CPC: {cpc_code}")

Patent 10000272 primary CPC: BB32B32B/B32B5/02


In [10]:
import requests
import time

# Read first 1000 patent IDs
with open('/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_ids.txt', 'r') as f:
    patent_ids = [line.strip() for line in f]

matches = 0

# Process patents
for original_id in patent_ids:
    # Clean ID (remove US and last 2 chars if letters)
    if original_id.startswith('US'):
        clean_id = original_id[2:]
        if any(c.isalpha() for c in clean_id[-2:]):
            clean_id = clean_id[:-2]
            
        # Make API call
        url = "https://api.patentsview.org/patents/query"
        query = {
            "q": {"patent_id": clean_id},
            "f": ["patent_id", "cpc_section_id", "cpc_subsection_id", "cpc_group_id", "cpc_subgroup_id"]
        }
        
        response = requests.post(url, json=query)
        data = response.json()
        
        # Extract and print CPC
        if data['patents'] and data['patents'][0]['cpcs']:
            primary_cpc = data['patents'][0]['cpcs'][0]
            cpc_code = f"{primary_cpc['cpc_section_id']}{primary_cpc['cpc_subsection_id']}{primary_cpc['cpc_group_id']}/{primary_cpc['cpc_subgroup_id']}"
            print(f"{original_id},{cpc_code}")
            matches += 1
            

                
        time.sleep(0.1)  # Rate limiting

print(f"\nTotal matches found: {matches}")

US10000083,BB41B41M/B41M5/3336
US10000272,BB32B32B/B32B5/02
US10000305,AA61A61F/A61F2/30
US10000432,CC07C07C/C07C29/50
US10000433,CC07C07C/C07C39/16
US10000438,AA23A23L/A23L27/86
US10000442,CC07C07C/C07C69/675
US10000443,AA61A61K/A61K31/09
US10000446,CC07C07C/C07C229/38
US10000447,AA61A61K/A61K9/127
US10000449,AA61A61K/A61K31/235
US10000450,AA61A61P/A61P1/00
US10000451,AA61A61K/A61K31/404
US10000452,CC07C07D/C07D215/56
US10000453,BB01B01J/B01J21/185
US10000454,AA61A61K/A61K31/192
US10000456,AA61A61P/A61P1/00
US10000457,CC07C07D/C07D233/60
US10000458,CC07C07D/C07D249/08
US10000459,AA61A61K/A61K31/415
US10000460,BB01B01J/B01J23/58
US10000461,CC07C07D/C07D317/36
US10000462,CC07C07D/C07D333/24
US10000463,AA61A61K/A61K31/381
US10000464,CC07C07D/C07D401/04
US10000465,AA61A61K/A61K31/4439
US10000466,AA61A61P/A61P7/02
US10000467,AA61A61K/A61K47/6871
US10000468,AA61A61K/A61K31/497
US10000469,AA61A61K/A61K31/497
US10000470,CC07C07D/C07D401/14
US10000471,AA61A61P/A61P1/04
US10000472,CC07C07D/C07D

KeyboardInterrupt: 

In [11]:
import requests
import time

# Read patent IDs
with open('/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_ids.txt', 'r') as f:
    patent_ids = [line.strip() for line in f]

matches = 0

# Open output file in append mode
with open('/home/matt/Proj/QSURv3/Data/Raw/PatentDataRaw/patcid/patent_cpc_output.csv', 'a') as outfile:
    # Write header
    outfile.write("patent_id,cpc\n")
    
    # Process patents
    for original_id in patent_ids:
        if original_id.startswith('US'):
            clean_id = original_id[2:]
            if any(c.isalpha() for c in clean_id[-2:]):
                clean_id = clean_id[:-2]
                
            # Make API call
            url = "https://api.patentsview.org/patents/query"
            query = {
                "q": {"patent_id": clean_id},
                "f": ["patent_id", "cpc_section_id", "cpc_subsection_id", "cpc_group_id", "cpc_subgroup_id"]
            }
            
            response = requests.post(url, json=query)
            data = response.json()
            
            # Extract and write CPC
            if data['patents'] and data['patents'][0]['cpcs']:
                primary_cpc = data['patents'][0]['cpcs'][0]
                cpc_code = f"{primary_cpc['cpc_section_id']}{primary_cpc['cpc_subsection_id']}{primary_cpc['cpc_group_id']}/{primary_cpc['cpc_subgroup_id']}"
                outfile.write(f"{original_id},{cpc_code}\n")
                outfile.flush()  # Force write to disk
                matches += 1
                print(f"Match {matches}: {original_id},{cpc_code}")  # Print to console to monitor progress
                
            time.sleep(0.1)  # Rate limiting

print(f"\nTotal matches found: {matches}")

Match 1: US10000083,BB41B41M/B41M5/3336
Match 2: US10000272,BB32B32B/B32B5/02
Match 3: US10000305,AA61A61F/A61F2/30
Match 4: US10000432,CC07C07C/C07C29/50
Match 5: US10000433,CC07C07C/C07C39/16
Match 6: US10000438,AA23A23L/A23L27/86
Match 7: US10000442,CC07C07C/C07C69/675
Match 8: US10000443,AA61A61K/A61K31/09
Match 9: US10000446,CC07C07C/C07C229/38
Match 10: US10000447,AA61A61K/A61K9/127
Match 11: US10000449,AA61A61K/A61K31/235
Match 12: US10000450,AA61A61P/A61P1/00
Match 13: US10000451,AA61A61K/A61K31/404
Match 14: US10000452,CC07C07D/C07D215/56
Match 15: US10000453,BB01B01J/B01J21/185
Match 16: US10000454,AA61A61K/A61K31/192
Match 17: US10000456,AA61A61P/A61P1/00
Match 18: US10000457,CC07C07D/C07D233/60
Match 19: US10000458,CC07C07D/C07D249/08
Match 20: US10000459,AA61A61K/A61K31/415
Match 21: US10000460,BB01B01J/B01J23/58
Match 22: US10000461,CC07C07D/C07D317/36
Match 23: US10000462,CC07C07D/C07D333/24
Match 24: US10000463,AA61A61K/A61K31/381
Match 25: US10000464,CC07C07D/C07D401/0

KeyboardInterrupt: 